**This program will parse your data of midi files into data readable by the ai**

Import

In [1]:
!pip install py_midicsv
!pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.3 MB/s eta 0:00:00


In [2]:
!rm -rf /content/music/

**Upload your zip file of midi files (all in root of zip) and rename to music.zip also create a folder under the same name in the content folder**

In [5]:
!unzip /content/music.zip -d /content/music/

Archive:  /content/music.zip
  inflating: /content/music/Civ rev v1.61.mid  


**Convert the midis into all one track**

In [61]:
#@title # **Convert the dataset and make sure to change midi dir if you changed it above** *Also* you may get errors but its probably ok
import py_midicsv as pm
import os
import re


midi_dir = "/content/music/" #@param {type:"string"}
midi_files = [f for f in os.listdir(midi_dir) if f.endswith('.mid')]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  out_instrument = "%"
  match instrument:
    case 0:
      out_instrument = "%"
    case 1:
      out_instrument = "^"
    case 2:
      out_instrument = "&"
    case 3:
      out_instrument = "*"
    case 4:
      out_instrument = "("
    case _:
      out_instrument = ")"


  dynamic = "$"
  if(velocity > 108):
    dynamic = "$"
  elif velocity > 74:
    dynamic = "#"
  elif velocity > 40:
    dynamic = "@"
  else:
    dynamic = "!"

  return [start, dynamic, length, out_instrument, note]

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
numNotesInCsv = 0
for midi_file in midi_files:
  csv_parsed = []
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    notesPlaying = []
    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Note_on_c":
          toAdd = True
          for i in notesPlaying:
            if(i[4] == split_csv_item[4]):
              toAdd = False
          if toAdd:
            notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          i = 0
          while i < len(notesPlaying):
            if(notesPlaying[i][4] == split_csv_item[4]):
              temp = get_note_event(notesPlaying[i][1], split_csv_item[4], split_csv_item[0], split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5])
              csv_parsed.append(temp)
              #csv_parsed.append(get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|')
              #csv_string += get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|'
              lastNoteStart = notesPlaying[i][1]
              notesPlaying.pop(i)
              i-=1
            i+=1
  except Exception as e:
    print(e)

  # Change to be delta time instead of abs and sort time
  csv_parsed = sorted(csv_parsed, key=lambda x: x[0])
  for i in range(len(csv_parsed) - 1):
    csv_item_copy = csv_parsed[i]
    csv_item_copy[0] = csv_parsed[i + 1][0] - csv_parsed[i][0]


    # Also concat to csv_parsed
    for j in csv_item_copy:
      csv_string += str(j)
    csv_string += "|"
    numNotesInCsv+=1
    if(numNotesInCsv % 400 == 0):
      csv_string += "\n"

#for i in range(0, len(csv_string), 1800):
#  csv_string[i] = "\n"

with open("convertedData.txt", "w") as f:
    f.writelines(csv_string)